In [1]:
import sys
import os

sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
import timm
from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform

import pandas as pd
import numpy as np
import glob
import cv2
import matplotlib.pyplot as plt
import joblib
import gc
from glob import glob

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

import timm
from PIL import Image
import PIL

from tqdm import tqdm
import joblib
import time
from tqdm.notebook import tqdm
import joblib
from sklearn.model_selection import StratifiedKFold

import cuml

print(np.__version__)
print(pd.__version__)
print(torch.__version__)
print(timm.__version__)

1.19.5
1.3.4
1.9.1
0.6.5


# Install OpenAI CLIP

In [2]:
!pip install ../input/openaiclipweights/python-ftfy-master/python-ftfy-master
!pip install ../input/openaiclipweights/clip/CLIP
!cp ../input/openaiclipweights/CLIP-main/CLIP-main/clip/bpe_simple_vocab_16e6.txt /opt/conda/lib/python3.7/site-packages/clip/.
!gzip -k /opt/conda/lib/python3.7/site-packages/clip/bpe_simple_vocab_16e6.txt

import clip

Processing /kaggle/input/openaiclipweights/python-ftfy-master/python-ftfy-master
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ftfy: filename=ftfy-6.0.2-py3-none-any.whl size=41932 sha256=ae6fa474ff0490aa847c04d05054e9f91ca55c0c40b5b22d4fc2b63b3bfae62a
  Stored in directory: /root/.cache/pip/wheels/2e/98/80/ce5445e83c31c177bd363169ed65e9cb14156696de5721bd36
Successfully built ftfy
Processing /kaggle/input/openaiclipweights/clip/CLIP
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your

# Load Train and Test

In [3]:
train = pd.read_csv('../input/petfinderdata/train-folds-1.csv')
test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
sub = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')

train['path'] = train['Id'].map(lambda x: '../input/petfinder-pawpularity-score/train/'+x+'.jpg')
test['path'] = test['Id'].map(lambda x: '../input/petfinder-pawpularity-score/test/'+x+'.jpg')

# If its Public LB run, then augment Testset to chack batch size memory consumption.
if test.shape[0]<10:
    test = pd.concat([
        test, test, test, test, test, 
    ])
    test = test.reset_index(drop=True)

print(train.shape, test.shape, sub.shape)

(9912, 18) (40, 14) (8, 2)


# Just create K Folds and check target consistency accross folds

In [4]:
train['bins'] = (train['Pawpularity']//5).round()

train['fold0'] = -1
skf = StratifiedKFold(n_splits = 20, shuffle=True, random_state = 1)
for i, (_, test_index) in enumerate(skf.split(train.index, train['bins'])):
    train.iloc[test_index, -1] = i

train['fold0'] = train['fold0'].astype('int')
gc.collect()

train.groupby(['fold0'])['Pawpularity'].agg(['mean','std','count'])

,mean,std,count
fold0,,,
0,38.044355,20.623269,496
1,38.016129,20.599788,496
2,38.018145,20.700209,496
3,38.028226,20.665865,496
4,38.026210,20.756338,496
5,38.016129,20.668127,496
6,38.070565,20.590600,496
7,38.030242,20.731401,496
8,38.125000,20.565454,496


In [5]:
train.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,fold,dup,nn0,path,bins,fold0
0,3ed899a8334a8e5c74f4a554f3ce6f08,0,1,1,1,0,0,0,0,0,0,0,0,1,1,-1,89a0b49aad4ce34c978b9a9c74cc95bd,../input/petfinder-pawpularity-score/train/3ed...,0,17
1,e0a1efdaf4fbed8659b6d23994ee346e,0,1,1,1,0,0,0,0,1,1,0,0,1,2,-2,b89199007b4fbd51864eeaf8f264f946,../input/petfinder-pawpularity-score/train/e0a...,0,14
2,6c159aede3df25fdbe781431aabcfc67,0,1,1,1,0,0,0,0,0,0,0,0,1,3,-3,59b3e4a3c4fe0f1f56cb7741100e7a30,../input/petfinder-pawpularity-score/train/6c1...,0,15
3,53b536999aecd800cfda720f3ca363cb,0,1,1,1,0,0,0,0,0,0,0,0,1,4,-4,0a05c55ca864b667d31c80ce2c68d6b3,../input/petfinder-pawpularity-score/train/53b...,0,19
4,4e3c8816d95b083b870c6747a26fcb58,0,0,0,1,0,0,0,0,1,1,0,1,2,5,-5,7af48bb7190c236d8876fe529b449c3f,../input/petfinder-pawpularity-score/train/4e3...,0,5


In [6]:
test.head()

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


# Lest check all models available in timm library

In [7]:
avail_pretrained_models = timm.list_models(pretrained=True)
len(avail_pretrained_models)

709

As we can see there are 709 pretrained model architectures available in timm library. Most of the models in timm are trained using 1000 classes in imagenet, so output shape is 1000 for each model.

The first part of the solution is basically extract the features from the last layer of that models and run a SVR on that extracted features. So the idea is to find a subset of models (from that 709) that performs well in terms of RMSE. To do that it was used a forward models selection algorithm, following by RMSE hill climbing logic. Starting with one model, then keep adding models until it stop increasing RMSE performance.

The pretrained models found by the forward model selection algorithm used in this notebook are listed below.

In [8]:
names = [
    'deit_base_distilled_patch16_384',
    'fbnetc_100',
    'ig_resnext101_32x8d',
    'ig_resnext101_32x48d',
    'repvgg_b0',
    'resnetv2_152x4_bitm',
    'rexnet_200',
    'resnest269e',
    'swsl_resnext101_32x8d',
    'tf_efficientnet_b6_ns',
    'tf_efficientnet_b7_ns',
    'tf_efficientnet_b8_ap',
    'tf_efficientnet_l2_ns_475',
    'vit_base_patch16_384',
    'vit_large_patch16_384',
    'vit_large_r50_s32_384',
]

names_hflip_crop = [
    'tf_efficientnet_l2_ns_hflip_384',
    'deit_base_distilled_patch16_384_hflip_384',
    'ig_resnext101_32x48d_hflip_384',
    'tf_efficientnet_l2_ns_512',
]

names_orig = [
    'ig_resnext101_32x48d',
    'vit_large_r50_s32_384',
    'clip_RN50x4',
    'clip_ViT-B-16',
    'clip_RN50x16',
    'clip_ViT-B-32',
]

# Create a dictionary with the path of all pretrained weights available in Kaggle datasets

In [9]:
modelpath = { m.split('/')[-1].split('.')[0] :m for m in glob('../input/pytorch-pretrained-0/*.pt')+glob('../input/pytorch-pretrained-1/*.pt')+glob('../input/pytorch-pretrained-2/*.pt')+glob('../input/pytorch-pretrained-3/*.pt')}
# modelpath

# Now interactively extract the TESTSET features from each imagenet pretrained model and append to a dictionary

In [10]:
class PawpularDataset:
    def __init__(self, images, base_path='../input/petfinder-pawpularity-score/train/', modelcfg=None, aug=0 ):
        
        self.images = images.copy()
        self.base_path = base_path
        self.transform = create_transform(**modelcfg)
        self.aug=aug
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, item):
        img = Image.open(self.base_path + self.images[item] + '.jpg').convert('RGB')
        img = self.transform(img)
        return img


EMB_TEST = {}
for arch in names:
    starttime = time.time()

    model = timm.create_model(arch, pretrained=False).to('cuda')
    model.load_state_dict(torch.load(modelpath[arch]))
    model.eval()

    train_dataset = PawpularDataset(
        images = test.Id.values,
        base_path='../input/petfinder-pawpularity-score/test/',
        modelcfg = resolve_data_config({}, model=model),
        aug = 0,
    )
    BS = 10 if arch in ['tf_efficientnet_l2_ns'] else 16
    train_dataloader = DataLoader(train_dataset, batch_size=BS, num_workers= 2, shuffle=False)
    
    with torch.no_grad():
        res = [model(img.to('cuda')).cpu().numpy() for img in train_dataloader]
    res = np.concatenate(res, 0)
    EMB_TEST[arch] = res
    
    print( arch, ', Done in:', int(time.time() - starttime), 's' )
    
    del model, res
    torch.cuda.empty_cache() # PyTorch thing to clean RAM
    gc.collect()

print(time.time() )    
len(EMB_TEST), EMB_TEST.keys()

deit_base_distilled_patch16_384 , Done in: 19 s
fbnetc_100 , Done in: 0 s
ig_resnext101_32x8d , Done in: 5 s
ig_resnext101_32x48d , Done in: 41 s
repvgg_b0 , Done in: 1 s
resnetv2_152x4_bitm , Done in: 51 s
rexnet_200 , Done in: 1 s
resnest269e , Done in: 9 s
swsl_resnext101_32x8d , Done in: 5 s
tf_efficientnet_b6_ns , Done in: 4 s
tf_efficientnet_b7_ns , Done in: 6 s
tf_efficientnet_b8_ap , Done in: 8 s
tf_efficientnet_l2_ns_475 , Done in: 28 s
vit_base_patch16_384 , Done in: 5 s
vit_large_patch16_384 , Done in: 17 s
vit_large_r50_s32_384 , Done in: 18 s
1701748208.097808


(16,
 dict_keys(['deit_base_distilled_patch16_384', 'fbnetc_100', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'repvgg_b0', 'resnetv2_152x4_bitm', 'rexnet_200', 'resnest269e', 'swsl_resnext101_32x8d', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_b8_ap', 'tf_efficientnet_l2_ns_475', 'vit_base_patch16_384', 'vit_large_patch16_384', 'vit_large_r50_s32_384']))

# Extract features using Horizontal Flip and small crop

In [11]:
class PawpularDataset_HFLIP:
    def __init__(self, images, base_path='../input/petfinder-pawpularity-score/train/', modelcfg=None, doflip=False ):
        
        self.images = images.copy()
        self.base_path = base_path
        self.transform = modelcfg
        self.doflip=doflip
        
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, item):
        img = Image.open(self.base_path + self.images[item] + '.jpg').convert('RGB')
        
        if self.doflip==True:
            img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            width, height = img.size
            img = img.crop((0.0*width, 0.02*height, 0.98*width, 0.98 * height))  
        
        img = self.transform(img)
        return img


for arch in names_hflip_crop:
    starttime = time.time()

    archname = arch.split('_hflip_')[0]
    if arch == 'tf_efficientnet_l2_ns_512':
        archname = 'tf_efficientnet_l2_ns'
    model = timm.create_model(archname, pretrained=False).to('cuda')
    model.load_state_dict(torch.load(modelpath[archname]))
    model.eval()

    # Get model default transforms
    transf = resolve_data_config({}, model=model)
    sz = int(arch.split('_')[-1])
    transf['input_size'] = (3, sz, sz)
    transf['crop_pct'] = 1.0        
    transf = create_transform(**transf)

    doflip = True if arch.split('_')[-2] == 'hflip' else False
    train_dataset = PawpularDataset_HFLIP(
        images = test.Id.values,
        base_path='../input/petfinder-pawpularity-score/test/',
        modelcfg = transf,
        doflip = doflip,
    )

    BS = 10 if archname in ['tf_efficientnet_l2_ns'] else 16
    train_dataloader = DataLoader(train_dataset, batch_size=BS, num_workers= 2, shuffle=False)

    with torch.no_grad():
        res = [model(img.to('cuda')).cpu().numpy() for img in train_dataloader]
    res = np.concatenate(res, 0)
    EMB_TEST[arch] = res

    print( arch, 'imge size:', sz, 'Hflip:', doflip, ',Done in:', int(time.time() - starttime), 's' )

    del model, res
    torch.cuda.empty_cache() # PyTorch thing to clean RAM
    gc.collect()

print(time.time() )    
len(EMB_TEST), EMB_TEST.keys()

tf_efficientnet_l2_ns_hflip_384 imge size: 384 Hflip: True ,Done in: 27 s
deit_base_distilled_patch16_384_hflip_384 imge size: 384 Hflip: True ,Done in: 2 s
ig_resnext101_32x48d_hflip_384 imge size: 384 Hflip: True ,Done in: 25 s
tf_efficientnet_l2_ns_512 imge size: 512 Hflip: False ,Done in: 14 s
1701748319.8385396


(20,
 dict_keys(['deit_base_distilled_patch16_384', 'fbnetc_100', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'repvgg_b0', 'resnetv2_152x4_bitm', 'rexnet_200', 'resnest269e', 'swsl_resnext101_32x8d', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_b8_ap', 'tf_efficientnet_l2_ns_475', 'vit_base_patch16_384', 'vit_large_patch16_384', 'vit_large_r50_s32_384', 'tf_efficientnet_l2_ns_hflip_384', 'deit_base_distilled_patch16_384_hflip_384', 'ig_resnext101_32x48d_hflip_384', 'tf_efficientnet_l2_ns_512']))

# Now extract TESTSET features from CLIP architecture

In [12]:
class CustomDataset:
    def __init__(self, data, base_path='../input/petfinder-pawpularity-score/test/', preprocess=None):
        
        self.data = data.copy()
        self.base_path = base_path
        if 'Pawpularity' not in self.data.columns:
            self.data['Pawpularity'] = 0
        self.preprocess=preprocess
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, item):
        img = Image.open(self.base_path + self.data.Id[item] + '.jpg').convert("RGB")
        img = self.preprocess(img)
        return img


for m in ['RN50', 'RN101', 'RN50x4', 'RN50x16', 'ViT-B-16', 'ViT-B-32']:
    starttime = time.time()
    model, preprocess = clip.load("../input/openaiclipweights/clip/CLIP/models/"+m+".pt")
    model.cuda().eval()
    
    EMB = []
    with torch.no_grad():
        test_dataset = CustomDataset(data = test, base_path='../input/petfinder-pawpularity-score/test/', preprocess=preprocess)
        test_data_loader = DataLoader(test_dataset, batch_size=64,num_workers=2,shuffle=False,pin_memory=True,)
        for batch in test_data_loader:
            image_features = model.encode_image(batch.to('cuda'))
            #image_features /= image_features.norm(dim=-1, keepdim=True)
            logits = image_features.cpu().numpy()
            EMB.append(logits)
    EMB = np.concatenate(EMB, 0)
    EMB = EMB.astype('float32')
    gc.collect()
    
    EMB_TEST['clip_'+m] = EMB
    print( m, ', Done in:', int(time.time() - starttime), 's' )
    
    del model
    torch.cuda.empty_cache() # PyTorch thing to clean RAM
    gc.collect()
    
gc.collect()
print(EMB_TEST.keys())

RN50 , Done in: 4 s
RN101 , Done in: 5 s
RN50x4 , Done in: 7 s
RN50x16 , Done in: 11 s
ViT-B-16 , Done in: 6 s
ViT-B-32 , Done in: 5 s
dict_keys(['deit_base_distilled_patch16_384', 'fbnetc_100', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'repvgg_b0', 'resnetv2_152x4_bitm', 'rexnet_200', 'resnest269e', 'swsl_resnext101_32x8d', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_b8_ap', 'tf_efficientnet_l2_ns_475', 'vit_base_patch16_384', 'vit_large_patch16_384', 'vit_large_r50_s32_384', 'tf_efficientnet_l2_ns_hflip_384', 'deit_base_distilled_patch16_384_hflip_384', 'ig_resnext101_32x48d_hflip_384', 'tf_efficientnet_l2_ns_512', 'clip_RN50', 'clip_RN101', 'clip_RN50x4', 'clip_RN50x16', 'clip_ViT-B-16', 'clip_ViT-B-32'])


# Load TRAINSET extracted features (made offline)

In [13]:
EMB_TRAIN = joblib.load('../input/petfinderdata/train-embeddings-direct-1.joblib')
gc.collect()

resclip = joblib.load('../input/openai-clip/train-embeddings-openai-clip-1.joblib')
for m in resclip.keys():
    EMB_TRAIN[m] = resclip[m]
del resclip
gc.collect()

hflipmodels = joblib.load('../input/petfinder-extracted-pretrained-1/extracted-pretrained-1.joblib')
for col in names_hflip_crop:
    EMB_TRAIN[col] = hflipmodels[col]
del hflipmodels
gc.collect()

print( len(EMB_TRAIN) )
print(EMB_TRAIN.keys())

58
dict_keys(['adv_inception_v3', 'cait_xs24_384', 'cspdarknet53', 'deit_base_distilled_patch16_384', 'deit_base_patch16_224', 'dla34', 'dm_nfnet_f2', 'dpn68', 'ese_vovnet19b_dw', 'fbnetc_100', 'hrnet_w18', 'hrnet_w30', 'ig_resnext101_32x8d', 'ig_resnext101_32x48d', 'mixnet_m', 'nfnet_l0', 'pit_ti_distilled_224', 'repvgg_b0', 'resnest101e', 'resnest200e', 'resnest269e', 'resnet34d', 'resnetv2_101x1_bitm', 'resnetv2_152x4_bitm', 'resnetv2_50x1_bitm', 'rexnet_130', 'rexnet_200', 'seresnet50', 'seresnet152d', 'ssl_resnext50_32x4d', 'swin_base_patch4_window7_224_in22k', 'swsl_resnext101_32x4d', 'swsl_resnext101_32x8d', 'tf_efficientnet_b5_ap', 'tf_efficientnet_b8_ap', 'tf_efficientnet_b6_ns', 'tf_efficientnet_b7_ns', 'tf_efficientnet_l2_ns_475', 'tf_efficientnetv2_l_in21ft1k', 'tf_efficientnet_l2_ns', 'tf_mixnet_s', 'twins_pcpvt_small', 'vit_base_patch16_384', 'vit_base_resnet50_384', 'vit_large_patch16_224', 'vit_large_patch16_384', 'vit_large_r50_s32_384', 'xception71', 'clip_RN50', 'cli

## Explore Heat Map

In [15]:
archname = 'swsl_resnext101_32x8d'
model = timm.create_model(archname, pretrained=False).to('cuda')
model.load_state_dict(torch.load(modelpath[archname]))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act2): ReLU(inplace=True)
      (aa): Identity()
      (conv3): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_

# Check the shape of the features

In [ ]:
for m in EMB_TEST.keys():
    print(EMB_TRAIN[m].shape, EMB_TEST[m].shape, m)

In [ ]:
names0 = [
    'clip_RN50x16',
    'clip_ViT-B-32',
    'clip_ViT-B-16',
    'clip_RN50x4',
    'deit_base_distilled_patch16_384',
    'ig_resnext101_32x48d',
    'repvgg_b0',
    'resnetv2_152x4_bitm',
    'swsl_resnext101_32x8d',
    'tf_efficientnet_l2_ns_475',
    'vit_base_patch16_384',
    'vit_large_r50_s32_384',
]

names1 = [
    'clip_RN50x16',
    'clip_RN101', 
    'clip_RN50',
    'fbnetc_100',
    'ig_resnext101_32x8d',
    'rexnet_200',
    'resnest269e',
    'tf_efficientnet_b6_ns',
    'tf_efficientnet_b8_ap',
    'tf_efficientnet_b7_ns',
    'vit_large_patch16_384',
]

names2 = [
    'tf_efficientnet_l2_ns_hflip_384',
    'deit_base_distilled_patch16_384_hflip_384',
    'ig_resnext101_32x48d_hflip_384',
    'tf_efficientnet_l2_ns_512',
    'ig_resnext101_32x48d',
    'vit_large_r50_s32_384',
    'clip_RN50x4',
    'clip_ViT-B-16',
    'clip_RN50x16',
    'clip_ViT-B-32',
]

names = np.unique(names0 + names1 + names2)
len(names), names

# Just clean memory of offline trainset features not going to be used here

In [ ]:
feats = list(EMB_TRAIN.keys())
for n in feats:
    if n not in names:
        del EMB_TRAIN[n]
        gc.collect()

# Now its time to fit a GPU accelerated SVR using cuml 

In [ ]:
from cuml.svm import SVR
from sklearn.preprocessing import StandardScaler

def fit_gpu_svr(TRAIN, TEST, kfoldcol='fold0'):
    
    ypredtrain_ = np.zeros(train.shape[0])
    ypredtest_ = np.zeros(test.shape[0])

    for fold in range(train[kfoldcol].max()+1):
        ind_train = train[kfoldcol] != fold
        ind_valid = train[kfoldcol] == fold

        model = SVR(C=16.0, kernel='rbf', degree=3, max_iter=4000, output_type='numpy')
        model.fit(TRAIN[ind_train], train.Pawpularity[ind_train].clip(1, 85)  )

        ypredtrain_[ind_valid] = np.clip(model.predict(TRAIN[ind_valid]), 1 , 100)
        ypredtest_ += np.clip(model.predict(TEST), 1, 100)

        del model
        gc.collect()

    ypredtest_ /= (train[kfoldcol].max()+1)

    return ypredtrain_, ypredtest_

def rmse(ytrue, ypred):
    return np.sqrt(np.mean((ytrue-ypred)**2))

# First, lets fit one SVR for each architecture independently
(not used in the submit version)

for col in names:
    
    TRAIN = EMB_TRAIN[col].copy()
    TEST = EMB_TEST[col].copy()

    scaler = StandardScaler()
    scaler.fit( np.vstack((TRAIN, TEST)) )
    TRAIN = scaler.transform(TRAIN)
    TEST = scaler.transform(TEST)
    
    ypredtrain, ypredtest = fit_gpu_svr(TRAIN, TEST, 'fold0')
    print(rmse(train.Pawpularity,ypredtrain), col)

As you can see above, features extracted from individual architectures have SVR RMSE ranging from 17.56 to 18.52.

# Concatenate some features and standardize

# Fit the SVR A using all K Folds.
# I noticed cliping the target in 85 slightly boosts RMSE.

In [ ]:
print('Concatenating:', names0)

TRAIN = np.concatenate([EMB_TRAIN[k] for k in names0], 1)
TEST = np.concatenate([EMB_TEST[k] for k in names0], 1)
scaler = StandardScaler()
scaler.fit( np.vstack((TRAIN, TEST)) )
gc.collect()

TRAIN = scaler.transform(TRAIN)
TEST = scaler.transform(TEST)
gc.collect()

# Check the output shape
print(TRAIN.shape, TEST.shape)

ypredtrainA, ypredtestA = fit_gpu_svr(TRAIN, TEST, 'fold0')
print(rmse(train.Pawpularity, ypredtrainA))

In [ ]:
del TRAIN, TEST
gc.collect()
torch.cuda.empty_cache()

# Also I noticed that using a multiplier of 1.032 boosts both CV and LB. It may be by the fact that SRV optimizes mean squared error and not RMSE.

In [ ]:
print('RMSE:', rmse(train.Pawpularity, 1.032*ypredtrainA))

# Now fit a SVR B using a second subset of features. The idea of fitting more subsets is to add diversity in posterior model ensemble and avoid the curse of dimensionality increasing too much the number of features.

In [ ]:
print('Concatenating:', names1)

TRAIN = np.concatenate([EMB_TRAIN[k] for k in names1], 1)
TEST = np.concatenate([EMB_TEST[k] for k in names1], 1)
scaler = StandardScaler()
scaler.fit( np.vstack((TRAIN, TEST)) )
gc.collect()

TRAIN = scaler.transform(TRAIN)
TEST = scaler.transform(TEST)
gc.collect()

print( TRAIN.shape, TEST.shape )

ypredtrainB, ypredtestB = fit_gpu_svr(TRAIN, TEST, 'fold0')
print('RMSE:', rmse(train.Pawpularity, ypredtrainB))

In [ ]:
del TRAIN, TEST
gc.collect()
torch.cuda.empty_cache()

# SVR C

In [ ]:
print('Concatenating:', names2)

TRAIN = np.concatenate([EMB_TRAIN[k] for k in names2], 1)
TEST = np.concatenate([EMB_TEST[k] for k in names2], 1)
scaler = StandardScaler()
scaler.fit( np.vstack((TRAIN, TEST)) )
gc.collect()

TRAIN = scaler.transform(TRAIN)
TEST = scaler.transform(TEST)
gc.collect()

print( TRAIN.shape, TEST.shape )

ypredtrainC, ypredtestC = fit_gpu_svr(TRAIN, TEST, 'fold0')
print('RMSE:', rmse(train.Pawpularity, ypredtrainC))

In [ ]:
del TRAIN, TEST
del EMB_TRAIN, EMB_TEST
gc.collect()

torch.cuda.empty_cache()
gc.collect()

# Optimize the overall RMSE using prediction of cuML SVR A, B, and C.

In [ ]:
from scipy.optimize import minimize

def min_func(K):
    ypredtrain = K[0]*ypredtrainA + K[1]*ypredtrainB + K[2]*ypredtrainC
    return rmse(train.Pawpularity, ypredtrain)
   
res = minimize(min_func, [1/3]*3, method='Nelder-Mead', tol=1e-6)
K = res.x
res

In [ ]:
ypredtrain = K[0]*ypredtrainA + K[1]*ypredtrainB + K[2]*ypredtrainC

test['Pawpularity'] = K[0]*ypredtestA + K[1]*ypredtestB + K[2]*ypredtestC

print('Ensemble weights:', K )
print('Final RMSE:', rmse(train.Pawpularity, ypredtrain) )

In [ ]:
test.head(8)

In [ ]:
test[['Id','Pawpularity']].to_csv('submission.csv', index=False)